In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00


In [2]:
import numpy as np
from datasets import load_dataset

In [3]:
dataset = load_dataset('cnn_dailymail', '1.0.0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [15]:
noun_phrases_custom = []

for index in range(0, 1000):
    text_custom = dataset['train'][index]['highlights']
    doc_custom = nlp(text)
    noun_phrases_custom.extend([f'Doc{index+1}: ' + chunk.text for chunk in doc_custom.noun_chunks])

print(noun_phrases_custom)

["Doc1: President Harding's illegitimate daughter", 'Doc1: couch', 'Doc1: Senate office', 'Doc1: Review', "Doc1: Harry Truman's daughter", 'Doc1: presidential threat', 'Doc1: reporter', "Doc1: President Taylor's daughter", 'Doc1: future president', 'Doc1: an enemy power', "Doc1: Woodrow Wilson's daughter", 'Doc1: a guru', 'Doc1: India', "Doc2: President Harding's illegitimate daughter", 'Doc2: couch', 'Doc2: Senate office', 'Doc2: Review', "Doc2: Harry Truman's daughter", 'Doc2: presidential threat', 'Doc2: reporter', "Doc2: President Taylor's daughter", 'Doc2: future president', 'Doc2: an enemy power', "Doc2: Woodrow Wilson's daughter", 'Doc2: a guru', 'Doc2: India', "Doc3: President Harding's illegitimate daughter", 'Doc3: couch', 'Doc3: Senate office', 'Doc3: Review', "Doc3: Harry Truman's daughter", 'Doc3: presidential threat', 'Doc3: reporter', "Doc3: President Taylor's daughter", 'Doc3: future president', 'Doc3: an enemy power', "Doc3: Woodrow Wilson's daughter", 'Doc3: a guru', 

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()
context_matrix = tfidf.fit_transform(noun_phrases_custom)

In [17]:
print(type(context_matrix))

<class 'scipy.sparse._csr.csr_matrix'>


In [18]:
tfidf.get_feature_names_out()

array(['an', 'couch', 'daughter', ..., 'truman', 'wilson', 'woodrow'],
      dtype=object)

In [23]:
def cosine_similarity_between_noun_phrases(np1, np2):
    np1_index = noun_phrases_custom.index(np1)
    np2_index = noun_phrases_custom.index(np2)
    cosine_sim = cosine_similarity(context_matrix[np1_index], context_matrix[np2_index])
    return cosine_sim[0][0]

noun_phrase_1 = "Doc1: President Harding's illegitimate daughter"
noun_phrase_2 = "Doc61: Senate office"
similarity = cosine_similarity_between_noun_phrases(noun_phrase_1, noun_phrase_2)
print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

noun_phrase_1 = "Doc1: President Harding's illegitimate daughter"
noun_phrase_2 = "Doc1: Harry Truman's daughter"
similarity = cosine_similarity_between_noun_phrases(noun_phrase_1, noun_phrase_2)
print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

noun_phrase_1 = "Doc1: President Harding's illegitimate daughter"
noun_phrase_2 = "Doc1: couch"
similarity = cosine_similarity_between_noun_phrases(noun_phrase_1, noun_phrase_2)
print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

Cosine similarity between 'Doc1: President Harding's illegitimate daughter' and 'Doc61: Senate office': 0.0
Cosine similarity between 'Doc1: President Harding's illegitimate daughter' and 'Doc1: Harry Truman's daughter': 0.6995386015254546
Cosine similarity between 'Doc1: President Harding's illegitimate daughter' and 'Doc1: couch': 0.7217616059659947
